In [4]:
import time
import numpy as np
from scipy import stats
import ipyparallel as ipp
import tensorflow as tf
from kmeans import Kmeans
from AggregatorFunc import snd_to_eng, wc_from_eng, global_update, bg_from_eng, belta_update, grad_update, delta_update, binary_search

In [5]:
from sklearn import datasets
iris = datasets.load_iris()
train = iris.data
target = iris.target

In [7]:
c = ipp.Client()
dview = c[:]

In [8]:
import os
dview.map(os.chdir, ['D:/GitHub/Parallel/classifier']*3)
print(dview.apply_sync(os.getcwd))

['D:\\GitHub\\Parallel\\classifier', 'D:\\GitHub\\Parallel\\classifier', 'D:\\GitHub\\Parallel\\classifier']


In [9]:
#create a directview
c[0]['X'] = train[:50]
c[1]['X'] = train[50:100]
c[2]['X'] = train[100:150]

c[0]['y'] = target[:50]
c[1]['y'] = target[50:100]
c[2]['y'] = target[100:150]

In [10]:
def random_init_w(k_cluster,X):
    describe = stats.describe(X)
    w = np.random.normal(loc = describe[2], scale =describe[3], size = (k_cluster,len(X[0])))    
    return w

In [11]:
cluster_number = 3
feature_number = train.shape[1]

In [12]:
t = 0
s = 0
b = 0 # aggregator consumption
R = 10

data_size = np.array([50,50,50])
gamma = 10
phi = 0.2
w_aggregator = random_init_w(3,train)
torque_aggregator = 1
stop = False

In [13]:
km = Kmeans(cluster_num = cluster_number)
dview.push(dict(Kmeans = Kmeans))
#sending LinearRegression object to engines
dview['km'] = km

In [14]:
while True:
        tic = time.time()
        w_aggregator = w_aggregator.reshape(cluster_number,feature_number)
        snd_to_eng(w_aggregator,torque_aggregator,c)
        t_0 = t
        t = t + torque_aggregator
        dview.execute("""
import numpy as np
import time  
from sklearn.metrics.pairwise import euclidean_distances
km.Rec_from_Agg(w_aggregator, torque_aggregator)
km.time_record()
if km.t > 0:
    km.Est_Belta(X)
km.fit(X,y)
        """)
        
        
        w_local, c_local = wc_from_eng(c,'km')
        #calculate local consumption per iteration
        c_per = np.array(c_local)/torque_aggregator
        w_aggregator = global_update(w_local, data_size)
        if stop:
            w_final = w_aggregator.reshape(cluster_number,feature_number)
            break
        #c_local.sum() equal to c*t
        #s = s + np.array(c_local).sum()+ b
        s = s + np.array(c_local).sum()+ 5*np.array(c_local).sum()
        
        if t_0 > 0:
            belta_local, grad_local = bg_from_eng(c,'km')
            belta_aggregator = belta_update(belta_local,data_size)
            grad_aggregator = grad_update(grad_local,data_size)
            delta_aggregator = delta_update(grad_local, grad_aggregator, data_size)
            torque_aggregator, G_list = binary_search(torque_aggregator, delta_aggregator, belta_aggregator, gamma, phi)
            print('New torque is:',torque_aggregator)
        
        toc = time.time()
        b = toc - tic
        temp = s + torque_aggregator*c_per.sum() + b
        if temp >= R:
            torque_max = (R-b-s)/c_per.sum()
            G_list = np.array(G_list)
            G_min = G_list.min()
            for i, item in enumerate(G_list):
                if item >= torque_max:
                    itme = G_min
            torque_aggregator = np.argmax(G_list) + 1
            stop = True 

D:\GitHub\Parallel\classifier\AggregatorFunc.py:179: RuntimeWarning: invalid value encountered in double_scalars
  h = delta* (pow((eta*belta + 1),torque)-1)/belta - eta*delta*torque


New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
got unknown result: 30a2bbd8-8ea86881ee167fa0b69c4ef2
New torque is: 1
New torque is: 1
got unknown result: f636fa9f-7711fb001cd66cc5f6b60e23
New torque is: 1


In [15]:
w_final

array([[ 5.89732409,  3.02189579,  2.55976898,  1.90917871],
       [ 5.45033144,  3.08237133,  5.47465109,  1.58563296],
       [ 5.23792976,  2.77056462, -3.28405904,  1.58761075]])

In [16]:
from sklearn.metrics.pairwise import euclidean_distances
a = euclidean_distances(train,w_final)

In [17]:
b = a.argmin(axis = 1)

In [18]:
b

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)